In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef, roc_auc_score, confusion_matrix
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

# Load the dataset
dataset = pd.read_excel('Final_non_redundant_sequences.xlsx', na_filter=False)
X_data_name = 'seqvev_whole_smaple_reduced_embeddings_file_ordered.csv'
X_data = pd.read_csv(X_data_name, header=0, index_col=0, delimiter=',')
X = np.array(X_data)
y = np.array(dataset['label'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def build_model(input_shape):
    input = Input(input_shape)
    x = Conv1D(64, 5, strides=1, padding='same')(input)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Dropout(0.25)(x)
    
    x = Conv1D(128, 5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Dropout(0.25)(x)
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input, outputs=x)
    return model

def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 10
    lr = initial_lr * np.power(drop, np.floor((1 + epoch) / epochs_drop))
    return lr

def train_model(X_train, y_train, X_test, y_test):
    input_shape = (1024, 1)
    model = build_model(input_shape)
    
    # Optimizer
    adam = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    lrate = LearningRateScheduler(step_decay)
    early_stop = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, restore_best_weights=True)
    mc = ModelCheckpoint('best_model_1024-seqvec.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    callbacks_list = [lrate, early_stop, mc]
    
    class_weight = {0: 1, 1: 2}  # Adjust the weights as needed
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=callbacks_list, batch_size=32, class_weight=class_weight)
    
    return model

# Train the model
trained_model = train_model(X_train, y_train, X_test, y_test)

# Load the best model
saved_model = load_model('best_model_1024-seqvec.h5')

# Function to optimize threshold based on MCC
def optimize_threshold(y_true, y_pred_probas):
    thresholds = np.arange(0.1, 1.0, 0.05)
    best_mcc = -1
    best_threshold = 0.5
    
    for threshold in thresholds:
        y_pred = (y_pred_probas > threshold).astype(int)
        mcc = matthews_corrcoef(y_true, y_pred)
        
        if mcc > best_mcc:
            best_mcc = mcc
            best_threshold = threshold
    
    return best_threshold, best_mcc




Epoch 1/100


137/137 [==============================] - ETA: 0s - loss: 21.9753 - accuracy: 0.8040
Epoch 1: val_accuracy improved from -inf to 0.25365, saving model to best_model_1024-seqvec.h5


C:\Users\Nandan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


137/137 [==============================] - 10s 67ms/step - loss: 21.9753 - accuracy: 0.8040 - val_loss: 6.9658 - val_accuracy: 0.2536 - lr: 0.0100
Epoch 2/100
137/137 [==============================] - ETA: 0s - loss: 0.5095 - accuracy: 0.8757
Epoch 2: val_accuracy did not improve from 0.25365
137/137 [==============================] - 9s 63ms/step - loss: 0.5095 - accuracy: 0.8757 - val_loss: 5.8722 - val_accuracy: 0.2536 - lr: 0.0100
Epoch 3/100
137/137 [==============================] - ETA: 0s - loss: 0.4749 - accuracy: 0.8852
Epoch 3: val_accuracy improved from 0.25365 to 0.25456, saving model to best_model_1024-seqvec.h5
137/137 [==============================] - 9s 66ms/step - loss: 0.4749 - accuracy: 0.8852 - val_loss: 1.7360 - val_accuracy: 0.2546 - lr: 0.0100
Epoch 4/100
137/137 [==============================] - ETA: 0s - loss: 0.4597 - accuracy: 0.8832
Epoch 4: val_accuracy improved from 0.25456 to 0.84945, saving model to best_model_1024-seqvec.h5
137/137 [================

In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, confusion_matrix
import math

k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=1)

# Convert back to DataFrames if needed (assuming X_train is still a numpy array)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# Result collection lists
ACC_collection = []
BACC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []

# Function to train the model for each fold
def ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV):
    # Train the model with the training fold
    model = train_model(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    return model

# Cross-validation loop
for train_index, test_index in kf.split(y_train):
    X_train_CV, X_valid_CV = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_train_CV, y_valid_CV = y_train.iloc[train_index], y_train.iloc[test_index]

    # Train the model for this fold
    model = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    
    # Load the best model
    saved_model = load_model('best_model_1024-seqvec.h5')
    
    # Predict probabilities
    predicted_probabilities = saved_model.predict(X_valid_CV, batch_size=1)
    
    # Convert probabilities to class predictions
    predicted_class = (predicted_probabilities > 0.5).astype(int)
    
    # Compute confusion matrix
    y_true = y_valid_CV.values
    TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel()
    
    # Compute metrics
    ACC = (TP + TN) / (TP + TN + FP + FN)
    Sn = TP / (TP + FN)
    Sp = TN / (TN + FP)
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    BACC = 0.5 * (Sn + Sp)
    AUC = roc_auc_score(y_true, predicted_probabilities)
    
    # Append metrics to collection lists
    ACC_collection.append(ACC)
    Sn_collection.append(Sn)
    Sp_collection.append(Sp)
    MCC_collection.append(MCC)
    BACC_collection.append(BACC)
    AUC_collection.append(AUC)

# Display the results for each fold
print("Average Accuracy:", np.mean(ACC_collection))
print("Average Balanced Accuracy:", np.mean(BACC_collection))
print("Average Sensitivity (Sn):", np.mean(Sn_collection))
print("Average Specificity (Sp):", np.mean(Sp_collection))
print("Average MCC:", np.mean(MCC_collection))
print("Average AUC:", np.mean(AUC_collection))

Epoch 1/100
123/124 [============================>.] - ETA: 0s - loss: 19.5462 - accuracy: 0.8067
Epoch 1: val_accuracy improved from -inf to 0.24601, saving model to best_model_1024-seqvec.h5


C:\Users\Nandan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


124/124 [==============================] - 9s 66ms/step - loss: 19.5090 - accuracy: 0.8068 - val_loss: 11.1627 - val_accuracy: 0.2460 - lr: 0.0100
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 0.4777 - accuracy: 0.8811
Epoch 2: val_accuracy did not improve from 0.24601
124/124 [==============================] - 7s 60ms/step - loss: 0.4779 - accuracy: 0.8811 - val_loss: 11.4327 - val_accuracy: 0.2460 - lr: 0.0100
Epoch 3/100
124/124 [==============================] - ETA: 0s - loss: 0.5040 - accuracy: 0.8816
Epoch 3: val_accuracy did not improve from 0.24601
124/124 [==============================] - 7s 60ms/step - loss: 0.5040 - accuracy: 0.8816 - val_loss: 6.0619 - val_accuracy: 0.2460 - lr: 0.0100
Epoch 4/100
123/124 [============================>.] - ETA: 0s - loss: 0.4430 - accuracy: 0.8905
Epoch 4: val_accuracy improved from 0.24601 to 0.24829, saving model to best_model_1024-seqvec.h5
124/124 [==============================] - 8s 66ms/step - loss: 0.4431 

C:\Users\Nandan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


124/124 [==============================] - 10s 71ms/step - loss: 34.5089 - accuracy: 0.8012 - val_loss: 7.1778 - val_accuracy: 0.2642 - lr: 0.0100
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 0.4934 - accuracy: 0.8707
Epoch 2: val_accuracy did not improve from 0.26424
124/124 [==============================] - 8s 64ms/step - loss: 0.4926 - accuracy: 0.8709 - val_loss: 6.0163 - val_accuracy: 0.2642 - lr: 0.0100
Epoch 3/100
123/124 [============================>.] - ETA: 0s - loss: 0.4650 - accuracy: 0.8829
Epoch 3: val_accuracy did not improve from 0.26424
124/124 [==============================] - 8s 63ms/step - loss: 0.4646 - accuracy: 0.8831 - val_loss: 1.4467 - val_accuracy: 0.2642 - lr: 0.0100
Epoch 4/100
123/124 [============================>.] - ETA: 0s - loss: 0.4424 - accuracy: 0.8864
Epoch 4: val_accuracy improved from 0.26424 to 0.57859, saving model to best_model_1024-seqvec.h5
124/124 [==============================] - 8s 67ms/step - loss: 0.4419 -

...

124/124 [==============================] - 9s 66ms/step - loss: 18.0904 - accuracy: 0.7853 - val_loss: 13.0270 - val_accuracy: 0.2694 - lr: 0.0100
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 0.5129 - accuracy: 0.8178
Epoch 2: val_accuracy did not improve from 0.26941
124/124 [==============================] - 7s 60ms/step - loss: 0.5124 - accuracy: 0.8183 - val_loss: 10.2315 - val_accuracy: 0.2694 - lr: 0.0100
Epoch 3/100
124/124 [==============================] - ETA: 0s - loss: 0.5255 - accuracy: 0.8674
Epoch 3: val_accuracy did not improve from 0.26941
124/124 [==============================] - 7s 60ms/step - loss: 0.5255 - accuracy: 0.8674 - val_loss: 9.0503 - val_accuracy: 0.2694 - lr: 0.0100
Epoch 4/100
123/124 [============================>.] - ETA: 0s - loss: 0.4819 - accuracy: 0.8686
Epoch 4: val_accuracy did not improve from 0.26941
124/124 [==============================] - 8s 60ms/step - loss: 0.4818 - accuracy: 0.8687 - val_loss: 3.8993 - val_acc

C:\Users\Nandan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


124/124 [==============================] - 9s 66ms/step - loss: 14.9165 - accuracy: 0.8058 - val_loss: 24.7411 - val_accuracy: 0.2374 - lr: 0.0100
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 0.4991 - accuracy: 0.8704
Epoch 2: val_accuracy did not improve from 0.23744
124/124 [==============================] - 8s 61ms/step - loss: 0.4984 - accuracy: 0.8707 - val_loss: 27.3735 - val_accuracy: 0.2374 - lr: 0.0100
Epoch 3/100
123/124 [============================>.] - ETA: 0s - loss: 0.4755 - accuracy: 0.8844
Epoch 3: val_accuracy did not improve from 0.23744
124/124 [==============================] - 8s 61ms/step - loss: 0.4754 - accuracy: 0.8844 - val_loss: 13.2927 - val_accuracy: 0.2374 - lr: 0.0100
Epoch 4/100
124/124 [==============================] - ETA: 0s - loss: 0.4305 - accuracy: 0.8905
Epoch 4: val_accuracy did not improve from 0.23744
124/124 [==============================] - 8s 61ms/step - loss: 0.4305 - accuracy: 0.8905 - val_loss: 2.4012 - val_ac

C:\Users\Nandan\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


124/124 [==============================] - 9s 66ms/step - loss: 13.5335 - accuracy: 0.7959 - val_loss: 10.0804 - val_accuracy: 0.2694 - lr: 0.0100
Epoch 2/100
123/124 [============================>.] - ETA: 0s - loss: 0.5112 - accuracy: 0.8681
Epoch 2: val_accuracy did not improve from 0.26941
124/124 [==============================] - 7s 60ms/step - loss: 0.5108 - accuracy: 0.8684 - val_loss: 12.4001 - val_accuracy: 0.2694 - lr: 0.0100
Epoch 3/100
123/124 [============================>.] - ETA: 0s - loss: 0.4812 - accuracy: 0.8717
Epoch 3: val_accuracy did not improve from 0.26941
124/124 [==============================] - 8s 61ms/step - loss: 0.4821 - accuracy: 0.8712 - val_loss: 8.7418 - val_accuracy: 0.2694 - lr: 0.0100
Epoch 4/100
124/124 [==============================] - ETA: 0s - loss: 0.4831 - accuracy: 0.8733
Epoch 4: val_accuracy did not improve from 0.26941
124/124 [==============================] - 8s 62ms/step - loss: 0.4831 - accuracy: 0.8733 - val_loss: 6.2256 - val_acc

In [3]:
# Display the results for each fold with mean and standard deviation
print("Accuracy: Mean =", np.mean(ACC_collection), ", Std =", np.std(ACC_collection))
print("Balanced Accuracy: Mean =", np.mean(BACC_collection), ", Std =", np.std(BACC_collection))
print("Sensitivity (Sn): Mean =", np.mean(Sn_collection), ", Std =", np.std(Sn_collection))
print("Specificity (Sp): Mean =", np.mean(Sp_collection), ", Std =", np.std(Sp_collection))
print("MCC: Mean =", np.mean(MCC_collection), ", Std =", np.std(MCC_collection))
print("AUC: Mean =", np.mean(AUC_collection), ", Std =", np.std(AUC_collection))

Accuracy: Mean = 0.9310980747027802 , Std = 0.008819153026379171
Balanced Accuracy: Mean = 0.8963021863779337 , Std = 0.015920291140749585
Sensitivity (Sn): Mean = 0.8247416341090554 , Std = 0.03385021172966556
Specificity (Sp): Mean = 0.9678627386468122 , Std = 0.007969565099501802
MCC: Mean = 0.8156231738295003 , Std = 0.023692977962399366
AUC: Mean = 0.9485315754786701 , Std = 0.018020429273061318


In [8]:
# Evaluate on the test dataset
predicted_probas_test = saved_model.predict(X_test, batch_size=32)
best_threshold_test, best_mcc_test = optimize_threshold(y_test, predicted_probas_test)
predicted_classes_test = (predicted_probas_test > best_threshold_test).astype(int)

# Calculate metrics for the test dataset with optimized threshold
accuracy_test = accuracy_score(y_test, predicted_classes_test)
sensitivity_test = recall_score(y_test, predicted_classes_test)  # Sensitivity (Recall)
TN_test, FP_test, FN_test, TP_test = confusion_matrix(y_test, predicted_classes_test).ravel()
specificity_test = TN_test / (TN_test + FP_test)  # Corrected Specificity calculation
MCC_test = matthews_corrcoef(y_test, predicted_classes_test)
auc_test = roc_auc_score(y_test, predicted_classes_test)

# Compute the correct balanced accuracy
balanced_accuracy_test = (sensitivity_test + specificity_test) / 2

# Print the adjusted results for the test dataset
print("\nOptimized Test Dataset Results:")
print(f"Accuracy (ACC): {accuracy_test}")
print(f"Balanced Accuracy (BACC): {balanced_accuracy_test}")
print(f"Sensitivity (Sn): {sensitivity_test}")
print(f"Specificity (Sp): {specificity_test}")
print(f"MCC: {MCC_test}")
print(f"AUC: {auc_test}")
print(f"True Positives (TP): {TP_test}")
print(f"False Positives (FP): {FP_test}")
print(f"True Negatives (TN): {TN_test}")
print(f"False Negatives (FN): {FN_test}")

# Print the total positive and total negative
total_positive = np.sum(y_test)
total_negative = len(y_test) - total_positive
print(f"Total Positive: {total_positive}")
print(f"Total Negative: {total_negative}")

35/35 [==============================] - 1s 23ms/step

Optimized Test Dataset Results:
Accuracy (ACC): 0.9324817518248175
Balanced Accuracy (BACC): 0.9013385868322457
Sensitivity (Sn): 0.8381294964028777
Specificity (Sp): 0.9645476772616137
MCC: 0.8188536545188286
AUC: 0.9013385868322458
True Positives (TP): 233
False Positives (FP): 29
True Negatives (TN): 789
False Negatives (FN): 45
Total Positive: 278
Total Negative: 818


In [9]:
# Evaluate on the external dataset (KELM)
dataset_external = pd.read_csv('kelm_dataset.csv', na_filter=False)
X_external_data_name = 'seqvev_kelm_data_reduced_embeddings_file_ordered.csv'
X_external_data = pd.read_csv(X_external_data_name, header=0, index_col=0, delimiter=',')
X_external = np.array(X_external_data)
y_external = np.array(dataset_external['label'])

# Normalize the external dataset
X_external_normalized = scaler.transform(X_external)

# Predict probabilities for external dataset
predicted_probas_ext = saved_model.predict(X_external_normalized, batch_size=32)
best_threshold_ext, best_mcc_ext = optimize_threshold(y_external, predicted_probas_ext)
predicted_classes_ext = (predicted_probas_ext > best_threshold_ext).astype(int)

# Calculate metrics for the external dataset with optimized threshold
accuracy_ext = accuracy_score(y_external, predicted_classes_ext)
sensitivity_ext = recall_score(y_external, predicted_classes_ext)  # Sensitivity (Recall)
TN_ext, FP_ext, FN_ext, TP_ext = confusion_matrix(y_external, predicted_classes_ext).ravel()
specificity_ext = TN_ext / (TN_ext + FP_ext)  # Corrected Specificity calculation
MCC_ext = matthews_corrcoef(y_external, predicted_classes_ext)
auc_ext = roc_auc_score(y_external, predicted_classes_ext)

# Compute the correct balanced accuracy
balanced_accuracy_ext = (sensitivity_ext + specificity_ext) / 2

# Print the adjusted results for the external dataset
print("\nOptimized External Dataset (KELM) Results:")
print(f"Accuracy (ACC): {accuracy_ext}")
print(f"Balanced Accuracy (BACC): {balanced_accuracy_ext}")
print(f"Sensitivity (Sn): {sensitivity_ext}")
print(f"Specificity (Sp): {specificity_ext}")
print(f"MCC: {MCC_ext}")
print(f"AUC: {auc_ext}")
print(f"True Positives (TP): {TP_ext}")
print(f"False Positives (FP): {FP_ext}")
print(f"True Negatives (TN): {TN_ext}")
print(f"False Negatives (FN): {FN_ext}")

# Print the total positive and total negative
total_positive_ext = np.sum(y_external)
total_negative_ext = len(y_external) - total_positive_ext
print(f"Total Positive: {total_positive_ext}")
print(f"Total Negative: {total_negative_ext}")

6/6 [==============================] - 0s 25ms/step

Optimized External Dataset (KELM) Results:
Accuracy (ACC): 0.8697916666666666
Balanced Accuracy (BACC): 0.8697916666666667
Sensitivity (Sn): 0.8020833333333334
Specificity (Sp): 0.9375
MCC: 0.7464591631114137
AUC: 0.8697916666666667
True Positives (TP): 77
False Positives (FP): 6
True Negatives (TN): 90
False Negatives (FN): 19
Total Positive: 96
Total Negative: 96
